In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import sys
from PIL import Image, ImageDraw


sys.path.append('/home/dan/work/cocoapi/PythonAPI/')
from pycocotools.coco import COCO

sys.path.append('../')
from detector.input_pipeline import ExtremePointsDataset

In [ ]:
IMAGES = '/home/dan/datasets/COCO/images/train2017/'
ANNOTATIONS = '/home/dan/datasets/COCO/annotations/person_keypoints_train2017.json'

In [ ]:
data = ExtremePointsDataset(
    COCO(ANNOTATIONS), image_folder=IMAGES, 
    is_training=True, training_size=640
)

# Get a training example

In [ ]:
%%time
image, heatmaps, offsets, masks, num_boxes = data[400]

In [ ]:
assert 5 * num_boxes == (heatmaps == 1).sum()
print('number of persons:', num_boxes)

In [ ]:
np.unique(masks.numpy())

# Show the image with masks

In [ ]:
# convert to the pil format
image = Image.fromarray((255*image.numpy()).astype('uint8'))
width, height = image.size
image.putalpha(255)

In [ ]:
def show_mask(image, mask):

    red = np.array([255, 0, 0], dtype='uint8')
    gray_mask = mask.numpy().astype('uint8')
    color_mask = red * np.expand_dims(gray_mask, 2)

    gray_mask = Image.fromarray(100 * gray_mask)
    color_mask = Image.fromarray(color_mask)
    color_mask.putalpha(gray_mask)
    
    image_copy = image.copy()
    image_copy.alpha_composite(color_mask.resize((width, height)))
    return image_copy

In [ ]:
show_mask(image, masks[0])

In [ ]:
show_mask(image, masks[1])

# Show the image with heatmaps

In [ ]:
image_copy = image.copy()

colors = {
    0: [255, 0, 0],  # red - top
    1: [0, 0, 255],  # blue - bottom
    2: [255, 255, 0],  # yellow - left
    3: [255, 0, 255],  # pink - right
    4: [0, 255, 0]  # green - center
}

for i, color in colors.items():

    h = np.expand_dims(heatmaps[i].numpy() > 1e-1, 2)
    gray_h = Image.fromarray(255*h[:, :, 0].astype('uint8'))
    color_h = Image.fromarray((color * h).astype('uint8'))
    
    color_h = color_h.resize((width, height))
    color_h.putalpha(gray_h.resize((width, height)))
    
    image_copy.alpha_composite(color_h)
    
image_copy

# Show the image with offsets

In [ ]:
stride = 4

true_peaks = {}
for i in range(5):
    
    h = heatmaps[i].numpy()
    y, x = np.where(h == 1)
    vectors = offsets[(2*i):(2*i + 2)].permute(1, 2, 0).numpy()
    assert (vectors[h < 1.0] == 0).all()
    
    offset_vectors = vectors[y, x]
    vy = offset_vectors[:, 0]
    vx = offset_vectors[:, 1]

    true_y = stride * (y + vy)
    true_x = stride * (x + vx)

    true_peaks[i] = np.stack([true_x, true_y], axis=1)

In [ ]:
image_copy = image.copy()
draw = ImageDraw.Draw(image_copy)
s = 3

for i, peaks in true_peaks.items():
    for x, y in peaks:
        color = tuple(colors[i])
        draw.ellipse([(x - s, y - s), (x + s, y + s)], fill=color)

image_copy